In [1]:
import pandas as pd
import json

# Nacteni dat

In [2]:
with open("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/movielens_1m/items.json",'r') as f:
    items = json.loads(f.read())

with open("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/movielens_1m/properties.json",'r') as f:
    properties = json.loads(f.read())
    
with open("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/movielens_1m/user.folds.json",'r') as f:
    user_folds = json.loads(f.read())
    
    
with open("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/movielens_1m/users.int2str.json",'r') as f:
    users_int2str = json.loads(f.read())
    
with open("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/movielens_1m/users.str2int.json",'r') as f:
    users_str2int = json.loads(f.read())
      
ratings = pd.read_csv('/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/movielens_1m/ratings.csv')

with open("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/movielens_1m/items.int2str.json",'r') as f:
    items_int2str = json.loads(f.read())
    
with open("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/movielens_1m/items.str2int.json",'r') as f:
    items_str2int = json.loads(f.read())



# Faktorizace

In [97]:
import numpy as np
%load_ext line_profiler

In [358]:
class MatrixFactorization:
    def __init__(self,users, items, ratings, no_features):
        self.users = users
        self.items = items
        self.no_users = len(users.keys())
        self.no_items = len(items.keys())
        self.ratings = dict(ratings.apply(lambda r:((r['userId'],r['itemId']),{'rating':r.rating}), axis=1).values)
        self.no_features = no_features
        self.init = False
        
        
    def compute_prediction(self, row):
        user_id = row['userId'] 
        item_id = row['itemId']
        
        try:
            prediction = sum(self.users[str(user_id)]['features'] * self.items[str(item_id)]['features'])
            return prediction
        except:
            return 
        
    def compute_error(self, row):
        rating = row['rating']
        prediction = row['prediction'] 
        try:
            return prediction - rating
        except:
            return
        
    '''
    INIT
    '''   
    def init_features(self, users_id, items_id):
        for user_id in users_id:
            self.users[user_id]['features'] = np.random.rand(self.no_features)

        for item_id in items_id:
            self.items[item_id]['features'] = np.random.rand(self.no_features)
        
        for key, content in self.ratings.items()
            self.ratings['prediction'] = self.ratings.apply(self.compute_prediction, axis=1)
            self.ratings['error'] = self.ratings.apply(self.compute_error, axis=1)

        self.init = True
        
    '''
    GRADIENT
    '''
    def compute_user_gradient(self, user_id, item_id):
#         error = self.ratings[(self.ratings['userId']==int(user_id)) & (self.ratings['itemId']==int(item_id)),'error']
        error = self.ratings.loc[(self.ratings['userId']==int(user_id)) & (self.ratings['itemId']==int(item_id))].loc[:,'error']
        if(error.empty):
            return 0
        else:
            print(error.values[0])
            return error.values[0] * self.users[user_id]['features'] + self.alfa * self.items[item_id]['features']

    def compute_item_gradient(self, user_id, item_id):
        error = self.ratings[(self.ratings['userId']==int(user_id)) & (self.ratings['itemId']==int(item_id)),'error']
        print(error,error.empty )
        if(error.empty):
            return 0
        else:
            return error * self.items[item_id]['features'] + self.alfa * self.users[user_id]['features']

          
    '''
    OPTIMAZE
    '''
    def rmse_optimalization(self, alfa):
        self.alfa = alfa
        users_id = self.users.keys()
        items_id = self.items.keys()

        #init features
        if(not self.init):
            self.init_features(users_id, items_id)
            print("Initialized features")
        
        for item_id in items_id:
            cumulated_gradient = 0
            for user_id in users_id:
                cumulated_gradient += -alfa*self.compute_user_gradient(user_id, item_id)
            self.items[item_id]['features'] += cumulated_gradient

        for user_id in users_id:
            cumulated_gradient = 0
            for item_id in items_id:
                cumulated_gradient += -alfa*self.compute_item_gradient(user_id, item_id)
            self.users[user_id]['features'] += cumulated_gradient


In [359]:
user_fold = user_folds[0]
users = list(users_str2int.keys())
users = dict([(u,{}) for u in users if not u in user_fold])
MFact = MatrixFactorization(users, items, ratings[:1000].copy(), 3)

In [360]:
MFact.ratings

{(1001, 3325): {'rating': -0.5},
 (1, 661): {'rating': 0.0},
 (10, 1918): {'rating': 0.0},
 (1001, 1844): {'rating': 1.0},
 (10, 2003): {'rating': 0.5},
 (1001, 3081): {'rating': 1.0},
 (1001, 3114): {'rating': 0.5},
 (10, 3928): {'rating': 0.5},
 (1001, 2333): {'rating': 0.5},
 (1, 2340): {'rating': 0.0},
 (1001, 1669): {'rating': 0.5},
 (10, 480): {'rating': 0.5},
 (10, 2109): {'rating': 0.5},
 (10, 1654): {'rating': 1.0},
 (1001, 3148): {'rating': -0.5},
 (10, 1954): {'rating': 0.0},
 (10, 1032): {'rating': 0.5},
 (1000, 3363): {'rating': 1.0},
 (1001, 3515): {'rating': 0.5},
 (10, 1380): {'rating': 1.0},
 (1, 1029): {'rating': 1.0},
 (10, 7): {'rating': 0.5},
 (10, 3309): {'rating': 0.0},
 (1001, 3158): {'rating': 0.5},
 (10, 1042): {'rating': 1.0},
 (10, 248): {'rating': 1.0},
 (1001, 2539): {'rating': -0.5},
 (10, 923): {'rating': 0.0},
 (1001, 1265): {'rating': 0.5},
 (1001, 2975): {'rating': 0.0},
 (10, 1271): {'rating': 1.0},
 (1001, 1573): {'rating': 0.5},
 (1001, 3192): {'ra

In [320]:
MFact.rmse_optimalization(0.001)

Initialized features
-0.0516559980777


KeyboardInterrupt: 

In [337]:
%lprun -f MFact.compute_user_gradient MFact.compute_user_gradient('1001', '608')

1.11309196805


In [ ]:
MFact.ratings

In [175]:
import numexpr